In [1]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup 
import pandas as pd
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.maximize_window()
driver.implicitly_wait(10)
driver.set_page_load_timeout(5)

In [2]:
df = pd.read_csv('match_results.csv')
link = df.sample(1)
link2 = link['score_card_link'].iloc[0]
driver.get(link2)

try:
    junk_click_box = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH,'//*[@id="wzrk-cancel"]'))
    )
    junk_click_box.click()
except NoSuchElementException:
    print('No Such element')

In [5]:
def process_batting_table(xpath):
    batting = driver.find_element(By.XPATH,xpath)
    batting_outer_html = batting.get_attribute('outerHTML')
    batting = pd.read_html(batting_outer_html)[0] 
    batting = (batting
                .dropna(axis = 0, how = 'all')
                .dropna(axis = 1, how = 'all')
                .rename(columns = {'Unnamed: 1':'Dismissal'})
    )
    # last 4 elements are junk and delete the last table
    batting = batting.iloc[:-4].dropna(axis = 1, how = 'all')
    batting_soup = BeautifulSoup(batting_outer_html)

    player_links = batting_soup.find_all(class_ = 'ds-inline-flex ds-items-start ds-leading-none') 

    player_links = [a['href'] for a in player_links]

    batting['player_links'] = player_links[:batting.shape[0]]
    
    return batting 


def process_bowling_table(xpath):
    bowling = driver.find_element(By.XPATH,xpath)
    bowling_outer_html = bowling.get_attribute('outerHTML')
    bowling = pd.read_html(bowling_outer_html)[0] 
    bowling = bowling[~bowling['BOWLING'].str.contains('\d', regex = True)]
    bowling = (bowling
                .dropna(axis = 0, how = 'all')
                .dropna(axis = 1, how = 'all')
    )

    bowling_soup = BeautifulSoup(bowling_outer_html)

    player_links = bowling_soup.find_all(class_ = 'ds-inline-flex ds-items-start ds-leading-none') 

    player_links = [a['href'] for a in player_links]

    bowling['player_links'] = player_links[:bowling.shape[0]]

    return bowling


In [6]:
team_one_batting_xpath = "(//table[@class='ds-w-full ds-table ds-table-md ds-table-auto  ci-scorecard-table'])[1]"
team_two_batting_xpath = "(//table[@class='ds-w-full ds-table ds-table-md ds-table-auto  ci-scorecard-table'])[2]"
team_one_bowling_xpath = '(//table[@class="ds-w-full ds-table ds-table-md ds-table-auto "])[1]'
team_two_bowling_xpath = '(//table[@class="ds-w-full ds-table ds-table-md ds-table-auto "])[2]'

In [7]:
# process batting 
batting_one_df = process_batting_table(team_one_batting_xpath)
batting_two_df = process_batting_table(team_two_batting_xpath)
batting_one_df['team_inning'] = link['team_one'].iloc[0]
batting_two_df['team_inning'] = link['team_two'].iloc[0]
batting_one_df['match_id'] = link['score_card'].iloc[0]
batting_two_df['match_id'] = link['score_card'].iloc[0]


In [11]:
# process bowling 
bowling_one_df = process_bowling_table(team_one_bowling_xpath)
bowling_two_df =process_bowling_table(team_two_bowling_xpath)
bowling_one_df['team_inning'] = link['team_one'].iloc[0]
bowling_two_df['team_inning'] = link['team_two'].iloc[0]
bowling_one_df['match_id'] = link['score_card'].iloc[0]
bowling_two_df['match_id'] = link['score_card'].iloc[0]

,BATTING,Dismissal,R,B,M,4s,6s,SR,player_links,team_inning,match_id
0,Paul Stirling,c Curran b Wood,14,8,16,1,1,175.00,/player/paul-stirling-303427,England,T20I # 1846
2,Andy Balbirnie (c),c Hales b Livingstone,62,47,74,5,2,131.91,/player/andy-balbirnie-303423,England,T20I # 1846
4,Lorcan Tucker †,run out (Rashid),34,27,42,3,1,125.92,/player/lorcan-tucker-928057,England,T20I # 1846
6,Harry Tector,c †Buttler b Wood,0,2,3,0,0,0.00,/player/harry-tector-961407,England,T20I # 1846
8,Curtis Campher,c †Buttler b Wood,18,11,22,3,0,163.63,/player/curtis-campher-595921,England,T20I # 1846
10,George Dockrell,b Livingstone,0,1,2,0,0,0.00,/player/george-dockrell-348034,England,T20I # 1846
12,Gareth Delany,not out,12,10,22,1,0,120.00,/player/gareth-delany-1099379,England,T20I # 1846
13,Mark Adair,c Curran b Livingstone,4,4,8,0,0,100.00,/player/mark-adair-526441,England,T20I # 1846
15,Barry McCarthy,b Curran,3,3,4,0,0,100.00,/player/barry-mccarthy-348059,England,T20I # 1846
17,Fionn Hand,b Curran,1,2,4,0,0,50.00,/player/fionn-hand-1099225,England,T20I # 1846
